In [1]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
train_path = "E:\mnist-in-csv\mnist_train.csv"
test_path = "E:\mnist-in-csv\mnist_test.csv"
train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

In [3]:
y_train = pd.DataFrame(train_data["label"])
y_test = pd.DataFrame(test_data["label"])
X_train = train_data.drop("label", axis = 1)
X_test = test_data.drop("label", axis = 1)

In [4]:
X_train.head()

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
def get_bat(df, x):
    return df[x:x+100]

In [6]:
X_test.head()

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
y_test.head()

,label
0,7
1,2
2,1
3,0
4,4


In [8]:
y_train.head()

,label
0,5
1,0
2,4
3,1
4,9


In [9]:
import torch
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms


In [10]:
X_train = torch.from_numpy(X_train.to_numpy())
X_test = torch.from_numpy(X_test.to_numpy())
y_train = torch.from_numpy(y_train.to_numpy())
y_test = torch.from_numpy(y_test.to_numpy())

In [11]:
# Build a feed-forward network
model = nn.Sequential(nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 10),
                      nn.Softmax(dim = 1))

# Define the loss
criterion = nn.CrossEntropyLoss()
# Forward pass, get our logits
logits = model(X_train.float())
y_train = y_train.reshape(60000)
logits = logits.reshape(60000, 10)
print(logits.size(), y_train.size())
# Calculate the loss with the logits and the labels
loss = criterion(logits, y_train)

print(loss)

torch.Size([60000, 10]) torch.Size([60000])
tensor(2.3307, grad_fn=<NllLossBackward>)


In [12]:
print('Before backward pass: \n', model[0].weight.grad)

loss.backward()

print('After backward pass: \n', model[0].weight.grad)

Before backward pass: 
 None
After backward pass: 
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])


In [13]:
from torch import optim

# Optimizers require the parameters to optimize and a learning rate
optimizer = optim.SGD(model.parameters(), lr=0.01)
print("Done")

Done


In [14]:
epochs = 50
for e in range(epochs):
    los = 0
    for i in range(0, 59000, 100):
        tf = get_bat(X_train, i)
        tl = get_bat(y_train, i)
        optimizer.zero_grad()
        output = model(tf.float())
        tl = tl.reshape(100)
        output = output.reshape(100, 10)
        loss = criterion(output, tl)
        loss.backward()
        optimizer.step()
        los += loss.item()
    print(e, " --", los/600)

0  -- 1.8903201540311179
1  -- 1.845658334493637
2  -- 1.7697433654467265
3  -- 1.727371614774068
4  -- 1.6681017323335012
5  -- 1.650929527680079
6  -- 1.5701249039173126
7  -- 1.5627571394046147
8  -- 1.5589066177606583
9  -- 1.5556218371788662
10  -- 1.5530705668528875
11  -- 1.5508847427368164
12  -- 1.5492331393559773
13  -- 1.5479921531677245
14  -- 1.5466222391525903
15  -- 1.5454421901702882
16  -- 1.544798062245051
17  -- 1.5438315371672313
18  -- 1.5430824154615401
19  -- 1.5423535776138306
20  -- 1.5418532651662826
21  -- 1.5414443600177765
22  -- 1.5409267071882884
23  -- 1.5406520048777261
24  -- 1.5402172966798147
25  -- 1.5399943248430887
26  -- 1.5396839108069738
27  -- 1.5394082359472911
28  -- 1.5080414628982544
29  -- 1.4531008340915044
30  -- 1.4509862504402797
31  -- 1.4493682195742925
32  -- 1.44816146393617
33  -- 1.4474577752749125
34  -- 1.4468246199687322
35  -- 1.446516167918841
36  -- 1.4461807743708293
37  -- 1.4460115792353947
38  -- 1.4457182401418687
39 

In [15]:
logits1 = model(X_test.float())
y_test = y_test.reshape(10000)
logits1 = logits1.reshape(10000, 10)
print(logits1.size(), y_test.size())
# Calculate the loss with the logits and the labels
loss = criterion(logits1, y_test)

print(loss)

torch.Size([10000, 10]) torch.Size([10000])
tensor(1.4862, grad_fn=<NllLossBackward>)


In [16]:
pr = 0
for i in range(len(y_test)):
    #print(y_test[i], logits[i])
    values, indices = logits1.max(1)
    #print(indices.size())
    if y_test[i] == indices[i]:
        pr += 1
print(pr/10000)

0.9754


In [17]:
import matplotlib.pyplot as plt
arr = [23, 5, 423, 64, 986, 263, 456, 87, 45, 392]
for i in arr:
    plt.figure()
    plt.imshow(X_test[i].reshape(28, 28))
    plt.title(indices[i])